In [1]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
video_files = glob.glob('video_footages/*')
video_files

['video_footages\\4K Road traffic video for object detection and tracking - free download now!.mp4',
 'video_footages\\Dutch bike rush hour this morning was smooth as ever.mp4',
 'video_footages\\Free City Street Footage - Royalty Free Stock Footage - People Walking Stock Footage No Copyright.mp4',
 'video_footages\\morning bike traffic during commute to Dutch university.mp4',
 'video_footages\\Raw Video_ Pittsburg Neighborhood Drive-By Shootings.mp4',
 'video_footages\\Road traffic video for object recognition.mp4',
 'video_footages\\Shooting captured by surveillance camera in Parma.mp4',
 'video_footages\\Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4',
 'video_footages\\Traffic Flow In The Highway - 4K Stock Videos _ NoCopyright _ AllVideoFree.mp4',
 'video_footages\\Unbelievably busy bicycle crossing in Amsterdam.mp4',
 'video_footages\\Utrecht Centraal Station Bike Parking.mp4',
 'video_footages\\vecteezy_busy-street-in-evening-hong

In [3]:
def iou(box1, box2):
    """
    the boxes should be in format of (x, y, w, h, area)
    """
    xx = max(box1[0],box2[0])
    yy = max(box1[1],box2[1])
    xf = min(box1[0]+box1[2],box2[0]+box2[2])
    yf = min(box1[1]+box1[3],box2[1]+box2[3])
    area1 = box1[2]*box1[3]
    area2 = box2[2]*box2[3]

    w = max(0, xf-xx)
    h = max(0, yf-yy)

    intersection = w*h
    union = abs(area1) + abs(area2) - intersection + 1e-6
    result = intersection/union
    return result

In [4]:
def nms(bboxes, threshold):
    assert type(bboxes) == list
    bboxes = [box for box in bboxes if box[-1] > threshold]
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
    bboxes_after_nms = []
    while bboxes:
        chosen_box = bboxes.pop(0)

        bboxes = [box for box in bboxes if iou(chosen_box[:4],box[:4])< threshold]

        bboxes_after_nms.append(chosen_box)

    return bboxes_after_nms

In [5]:
def get_mask(frame1, frame2, kernel=np.array((9,9), dtype=np.uint8)):

    frame_diff = cv2.subtract(frame2, frame1)

    # blur the frame difference
    frame_diff = cv2.medianBlur(frame_diff, 3)
    
    mask = cv2.adaptiveThreshold(frame_diff, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY_INV, 11, 3)

    mask = cv2.medianBlur(mask, 3)

    # morphological operations
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)

    return mask

In [8]:
cap = cv2.VideoCapture(video_files[-7])

thresh = 1000

ret, old_frame = cap.read()
height, width, layers = old_frame.shape
new_h = height / 2
new_w = width / 2
motion_thresh = np.c_[np.linspace(0.3, 1, int(new_h))].repeat(int(new_w), axis=-1)
old_frame = cv2.resize(old_frame, (int(new_w), int(new_h)))
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

while(cap.isOpened()):
    
    ret, frame = cap.read()
    if ret == True:
      
        frame = cv2.resize(frame, (int(new_w), int(new_h)))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        delta_frame = get_mask(old_gray, gray)

        contours, _ = cv2.findContours(delta_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_L1)

        predictions = []
        for cnt in contours:
            x,y,w,h = cv2.boundingRect(cnt)
            area = w*h
            predictions.append([x,y,w,h,area])
        predictions = nms(predictions, thresh)
        for box in predictions:
            #if box[-1] > thresh:
            frame = cv2.rectangle(frame, (box[0], box[1]), (box[0]+box[2], box[1]+box[3]), (0, 255, 0), thickness = 2)

            
        cv2.imshow('normal video',frame)
        cv2.imshow('difference video', delta_frame)


        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

        old_gray = gray.copy()

    else:
        break

        
 
cap.release()
 
cv2.destroyAllWindows()